# Stage 1: Identify Continues Blocks of CpGs that are Correlated (R^2 >= 0.5)

In [46]:
library(dplyr)
library(tidyr)
library(matrixStats)
library(stringr)

create_empty_table <- function(num_rows, num_cols) {
  frame <- data.frame(matrix(NA, nrow = num_rows, ncol = num_cols))
  return(frame)
}
as.numeric.factor <- function(x) {as.numeric(levels(x))[x]}


min.corr <- 0.707107

#Input
chr_num = "19"# args[1]
merged_coverage_files = paste("../input_files/gtex_raw_bins/chr",chr_num,"/ind",sep="")
#tissue_coverage_files = paste("../input_files/gtex_raw_bins/chr",chr_num,"/tissue/",sep="")


#file1 <- read.csv("./chr1/meth_coverage/bin_200bp/ind/13OW7.txt.gathered.bam.sorted.bismark.bins.csv",header = T)
#x = "./input_files/gtex_raw_bins/chr5/ind/X261.txt.gathered.bam.sorted.bismark.cov.cyt.merged_CpG_evidence.bins.csv"
#x = "../input_files/gtex_raw_bins/chr1/ind/chr1.X261.txt.gathered.bam.sorted.bismark.cov.cyt.merged_CpG_evidence.bins.csv"
#strsplit(strsplit(x,'\\.')[[1]][3],"/")[[1]][6]
#strsplit(x,'\\.')[[1]][4]

file_names <- dir(merged_coverage_files,full.names = T) 
df <- do.call(rbind,
            lapply(file_names, 
                   function(x) cbind(read.csv(x),
                                     #name=strsplit(strsplit(x,'\\.')[[1]][2],"/")[[1]][6]
                                     name=strsplit(x,'\\.')[[1]][4]
                   )
            )
)




In [47]:
head(df)

UCSC.Browser.Coordinates,Bin.Name,Avg.Methylation,CpGs.Covered,Num.of.CpGs.with.RD_n,name
<fct>,<fct>,<dbl>,<int>,<int>,<fct>
chr19:68500-68600,chr19_68600,96.491228,3,3,13OW7
chr19:70200-70300,chr19_70300,1.123596,3,3,13OW7
chr19:80600-80700,chr19_80700,85.000000,2,2,13OW7
chr19:80900-81000,chr19_81000,92.063492,2,2,13OW7
chr19:81600-81700,chr19_81700,62.857143,2,2,13OW7
chr19:91000-91100,chr19_91100,69.285714,2,2,13OW7


In [50]:
individuals <- as.character(unique(df$name))
I <- split(df, df$name)
combined_I <- I %>% Reduce(function(dtf1,dtf2) full_join(dtf1,dtf2,by="Bin.Name"), .)
combined_I <- na.omit(combined_I)
avg.m<- combined_I[ , grepl( "Avg.Methy" , names( combined_I ))]
combined_I$IIV <- rowVars(as.matrix(avg.m))
temp_min_max <- as.data.frame(rowRanges(as.matrix(avg.m),na.rm = TRUE ))
combined_I$IIR <- abs(temp_min_max$V1-temp_min_max$V2)
sortedcombined_I <- combined_I[order(-combined_I$IIR),]#sorted by the range
sortedcombined_I <-na.omit(sortedcombined_I)
avg.methy_df_ind <- sortedcombined_I[,c(grep("Bin.|Avg.",names(sortedcombined_I))), drop=F]
colnames(avg.methy_df_ind)[2:11] <- individuals

#combined_I$IIV <- row(combined_I[ , grepl( "Avg.Methy" , names( combined_I ) ) ], na.rm=TRUE)

#write.csv(x = sortedcombined_I,file = "temp_stage1.csv",row.names = F)
#extract top %20 of the bins
avg.methy_df_ind <- sortedcombined_I[,c(grep("Bin.|Avg.",names(sortedcombined_I))), drop=F]

colnames(avg.methy_df_ind)[2:11] <- individuals


split_Bin.name <- data.frame(str_split_fixed(avg.methy_df_ind$Bin.Name, "_", 2))
avg.methy_df_ind <- cbind(avg.methy_df_ind,split_Bin.name)
avg.methy_df_ind$X1 <- NULL
avg.methy_df_ind$X2 <- as.numeric(levels(avg.methy_df_ind$X2))[avg.methy_df_ind$X2]
avg.methy_df_ind <- avg.methy_df_ind[order(avg.methy_df_ind$X2),]

#avg.methy_df_ind <- avg.methy_df_ind[1:100,]

number_of_bins <- dim(avg.methy_df_ind)[1]
rownames(avg.methy_df_ind) <- NULL


i=1
min.dist <- 200
collect_index <- c()
cluster_num <- 0
cluster_vec <- c()
                           
while(i < number_of_bins){
    #print(i)
    rec_i <- avg.methy_df_ind[i,]
    rec_j <- avg.methy_df_ind[i+1,]

    if(as.numeric(rec_j[,12])-as.numeric(rec_i[,12]) > min.dist){
        i <- i + 1
        next
    }

    vec1 <- as.numeric(rec_i[,c(2:11)])
    vec2 <- as.numeric(rec_j[,c(2:11)])
    pair_corr <- try(
    expr = cor(vec1,vec2,use = "pairwise.complete.obs",method ="pearson")
    )
    if(is.na(pair_corr)){pair_corr <- 0}

    if(pair_corr < min.corr){i <- i + 1 
                             next }


    if(i %in% collect_index ){
        cluster_vec <- c(cluster_vec,rep(cluster_num,2))
    }else{
        cluster_num <- cluster_num + 1
        cluster_vec <- c(cluster_vec,rep(cluster_num,2))
    }
    collect_index <- c(collect_index,c(i,i+1))
    i <- i + 1
}


avg.methy_df_ind_clustered <- avg.methy_df_ind[collect_index,]
avg.methy_df_ind_clustered$cluser_ID <- cluster_vec
avg.methy_df_ind_clustered <- avg.methy_df_ind_clustered[!duplicated(avg.methy_df_ind_clustered$Bin.Name),]

write.csv(avg.methy_df_ind_clustered,file = paste("../predefined_R2_0.5/chr",chr_num,"_",min.corr,".csv",sep=""),row.names = F)
  

Warning message in cor(vec1, vec2, use = "pairwise.complete.obs", method = "pearson"):
“the standard deviation is zero”Warning message in cor(vec1, vec2, use = "pairwise.complete.obs", method = "pearson"):
“the standard deviation is zero”Warning message in cor(vec1, vec2, use = "pairwise.complete.obs", method = "pearson"):
“the standard deviation is zero”Warning message in cor(vec1, vec2, use = "pairwise.complete.obs", method = "pearson"):
“the standard deviation is zero”Warning message in cor(vec1, vec2, use = "pairwise.complete.obs", method = "pearson"):
“the standard deviation is zero”

In [53]:
head(avg.methy_df_ind_clustered,20)

,Bin.Name,13OW7,13QIC,14PQA,1F52S,1F6RS,1GZHY,RVPV,S7SE,WVLH,X261,X2,cluser_ID
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
17,chr19_93300,97.663551,87.434555,83.934426,91.883117,92.1296296,93.055556,90.714286,93.225806,92.727273,89.944134,93300,1
18,chr19_93400,95.753425,89.236111,85.787848,91.658391,88.8888889,94.456763,88.396349,94.905386,92.625899,87.357631,93400,1
19,chr19_93500,93.958076,87.735849,86.235187,91.107078,90.6647808,93.159609,88.061336,92.307692,89.825971,87.860697,93500,1
20,chr19_93600,84.641256,74.688797,79.239130,81.589537,82.3899371,80.529595,81.733021,87.648456,83.445491,74.206755,93600,2
21,chr19_93700,95.087719,92.264151,93.189557,93.776371,95.0067476,95.432300,94.276795,96.828992,92.571429,92.045455,93700,2
85,chr19_200400,95.864662,96.047431,92.401216,93.763920,95.2112676,93.582888,89.825581,94.301994,97.487437,94.662921,200400,3
86,chr19_200500,87.000000,84.615385,74.887892,73.228346,77.4774775,87.500000,68.103448,80.314961,89.855072,77.894737,200500,3
126,chr19_240700,84.023669,81.280788,84.444444,82.800000,73.3766234,86.187845,84.516129,89.603960,80.158730,85.500000,240700,4
127,chr19_240800,87.037037,91.208791,94.160584,88.957055,85.6164384,93.491124,90.000000,93.750000,87.387387,91.860465,240800,4


# Stage 2: Calculate Inter-Tissue_Correlations in identified Blocks

In [ ]:
# import libraries and load all data
import sys
import pandas as pd
import numpy as np
import os
from scipy import stats
import glob
from itertools import chain

chromosome="chr19" #sys.argv[1]

path = '../input_files/gtex_raw_covs/'+chromosome+'/*.cov'
print(path)

def get_data(path):
    cov_files = glob.glob(path)
    df = pd.DataFrame()
    for file_ in cov_files:
        file_df = pd.read_csv(file_,sep='\t',header=None )
        file_df['file_name'] = "-".join(os.path.basename(file_).split(".")[0].split("-")[0:3])
        df = df.append(file_df)

    sample_name_anno = pd.read_csv("../annotations/sample_name_annotation.csv",header = None)
    sample_name_anno.columns = ["sample_name","tissue_type"]
    df.columns = ["chr","start","end","perc","nMeth","nUnmeth","sample_name"]
    individuals = list(set([sample_name_anno.sample_name[i].split("-")[1] for i in range(0,30)]))

    all_sample_df = pd.merge(df,sample_name_anno,on = "sample_name")

    #get methylation values for the block in 10 thyroid samples
    Heart = all_sample_df[all_sample_df['tissue_type']=="Heart"]
    Brain = all_sample_df[all_sample_df['tissue_type']=="Brain"]
    Thyroid = all_sample_df[all_sample_df['tissue_type']=="Thyroid"]
    return([Heart,Brain,Thyroid,individuals])


def get_corr(start,end):
    heart_values = []
    brain_values = []
    thyroid_values = []
    sub_h = Heart[(start <= Heart['start']) & (Heart['end'] <= end)]
    sub_b = Brain[(start <= Brain['start']) & (Brain['end'] <= end)]
    sub_t = Thyroid[(start <= Thyroid['start']) & (Thyroid['end'] <= end)]


    for ind in individuals:
        temp1 = sub_h[sub_h['sample_name'].str.contains(ind)]
        heart_values.append(get_meth(temp1))

        temp2 = sub_b[sub_b['sample_name'].str.contains(ind)]
        brain_values.append(get_meth(temp2))

        temp3 = sub_t[sub_t['sample_name'].str.contains(ind)]
        thyroid_values.append(get_meth(temp3))


    block_methylation_df = pd.DataFrame(
    {'heart': heart_values,
     'brain': brain_values,
     'thyroid': thyroid_values
    })
    block_methylation_df = block_methylation_df[['heart','brain','thyroid']]
    cor_pairs = block_methylation_df.corr(method='pearson', min_periods=7)
    return([block_methylation_df,cor_pairs])

def get_meth(block_sample):
    if(np.ceil(block_sample.shape[0]/2) < block_sample[(block_sample['nMeth'] + block_sample['nUnmeth']) >=5].shape[0]):
        return(block_sample['perc'].mean())
    else:
        return (np.nan)


[Heart,Brain,Thyroid,individuals] = get_data(path)
for i in [0.707107]:
    corr = str(i)
    blocks = pd.read_csv("../predefined_R2_0.5/"+chromosome+"_"+corr+".csv")
    blocks['cluser_ID'] = blocks['cluser_ID'].astype(np.int64)
    h_b_list = []
    h_t_list = []
    b_t_list = []
    for i in range(1,blocks['cluser_ID'].iloc[-1]):
        #print(i)
        temp_range = blocks[blocks['cluser_ID']==i].X2
        start = temp_range.iloc[0]-100
        end = temp_range.iloc[-1]
        temp_corr_mat=get_corr(start,end)[1]
        h_b_list.append(temp_corr_mat.iloc[0,1])
        h_t_list.append(temp_corr_mat.iloc[0,2])
        b_t_list.append(temp_corr_mat.iloc[1,2])  

    ITC_DF = pd.DataFrame.from_dict({'block_id':list(range(1,i+1)),
                                 'Heart_Brain':h_b_list,
                                 'Heart_Thyroid':h_t_list,
                                 'Brain_Thyroid':b_t_list},
                               )

    ITC_DF = ITC_DF[['block_id','Heart_Brain','Heart_Thyroid','Brain_Thyroid']]

    ITC_DF.to_csv("../predefined_R2_0.5/"+chromosome+"_"+corr+"_ITC.csv")


